### Lista de Exercício 1

In [4]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import contextily
import folium
import geopandas

In [5]:
#### Carregando os dados
# Importando "listings data"
url_listings = "https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2024-12-27/data/listings.csv.gz"
df_listings = pd.read_csv(url_listings)

# Importando "calendar data"
url_calendar = "https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2024-12-27/data/calendar.csv.gz"
# OBS: Vamos chamar o arquivo de "df_prices" para facilitar a manipulação
df_prices = pd.read_csv(url_calendar, compression="gzip")

In [8]:
df_prices['price'].dtype

dtype('float64')

In [7]:
df_prices['price'] = df_prices['price'].str.replace('[$,]', '', regex=True).astype(float)

In [9]:
df_prices['price'].dtype

dtype('float64')

In [10]:
df_prices['date'] = pd.to_datetime(df_prices['date'], format='%Y-%m-%d')

In [11]:
df_prices['available']

0           f
1           f
2           f
3           f
4           f
           ..
14416834    t
14416835    t
14416836    t
14416837    t
14416838    t
Name: available, Length: 14416839, dtype: object

In [12]:
df_prices['available'] = df_prices['available']=='f'

In [13]:
df_prices['available']

0            True
1            True
2            True
3            True
4            True
            ...  
14416834    False
14416835    False
14416836    False
14416837    False
14416838    False
Name: available, Length: 14416839, dtype: bool

In [14]:
# Cleaning listings data
df_listings['price'] = df_listings['price'].str.replace('[$,]', '', regex=True).astype(float)
df_listings = df_listings[['id', 'name', 'neighbourhood_cleansed', 'latitude', 'longitude', 'price',
                               'host_id', 'host_name', 'host_response_time', 'host_response_rate', 'host_acceptance_rate',
                               'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities',
                               'number_of_reviews', 'reviews_per_month', 'first_review', 'last_review', 'review_scores_rating']]


In [15]:
df_listings = df_listings.rename(columns={'id': 'listing_id',
                                              'name': 'listing_title',
                                              'price': 'mean_price',
                                              'neighbourhood_cleansed': 'neighborhood'})

In [16]:
df_prices

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2024-12-28,True,350.0,NaN,5.0,28.0
1,17878,2024-12-29,True,350.0,NaN,5.0,28.0
2,17878,2024-12-30,True,350.0,NaN,5.0,28.0
3,17878,2024-12-31,True,350.0,NaN,5.0,28.0
4,17878,2025-01-01,True,350.0,NaN,5.0,28.0
...,...,...,...,...,...,...,...
14416834,1319654458578424849,2025-12-23,False,1350.0,NaN,3.0,14.0
14416835,1319654458578424849,2025-12-24,False,1350.0,NaN,3.0,14.0
14416836,1319654458578424849,2025-12-25,False,1350.0,NaN,3.0,14.0
14416837,1319654458578424849,2025-12-26,False,1350.0,NaN,3.0,14.0


In [17]:
df_listings

,listing_id,listing_title,neighborhood,latitude,longitude,mean_price,host_id,host_name,host_response_time,host_response_rate,...,accommodates,bathrooms,bedrooms,beds,amenities,number_of_reviews,reviews_per_month,first_review,last_review,review_scores_rating
0,271975,Unbelievable Ocean View Apartment,Vidigal,-22.995080,-43.236040,236.0,1423140,Maria,a few days or more,0%,...,4,1.0,2.0,2.0,"[""Essentials"", ""Shampoo"", ""Washer"", ""Smoking a...",7,0.05,2013-02-16,2019-10-05,4.14
1,272335,EXCELENTE APARTAMENTO EM COPACABANA,Copacabana,-22.962300,-43.173890,723.0,7506316,Vitor,NaN,NaN,...,4,1.0,1.0,1.0,"[""Essentials"", ""Cooking basics"", ""32 inch HDTV...",367,2.31,2011-12-06,2024-12-14,4.92
2,272631,Comfort em Copacabana,Copacabana,-22.967500,-43.186660,1146.0,1426830,Neyde,within an hour,100%,...,2,1.0,1.0,2.0,"[""Essentials"", ""Long term stays allowed"", ""Sha...",32,0.25,2014-07-01,2024-05-06,5.00
3,273261,Amazing Huge Apartment in Leblon,Leblon,-22.986320,-43.229360,6742.0,533566,Lucas,NaN,NaN,...,6,3.5,3.0,3.0,"[""Kitchen"", ""Dryer"", ""Washer"", ""Fire extinguis...",0,NaN,NaN,NaN,NaN
4,273363,Lindo quarto sossegado para temporada,Santa Teresa,-22.918250,-43.178720,61.0,556738,Casa Da Carmen E,within a few hours,100%,...,1,1.0,1.0,1.0,"[""Bay view"", ""Cleaning available during stay"",...",5,0.04,2015-03-11,2024-06-22,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39494,1319593870376466401,Copa Reformado 1 qrt perto do metrô,Copacabana,-22.969288,-43.188730,957.0,17325065,Artur,within an hour,100%,...,4,2.0,1.0,2.0,"[""Coffee maker"", ""Washer"", ""Hair dryer"", ""Bed ...",0,NaN,NaN,NaN,NaN
39495,1319611826461021550,Casa Sabiá,Jacarezinho,-22.889180,-43.264740,288.0,476519421,Bianca,NaN,NaN,...,8,1.0,1.0,1.0,"[""TV"", ""Outdoor dining area"", ""Pets allowed"", ...",0,NaN,NaN,NaN,NaN
39496,1319620401959854025,Apartamento na Barata Ribeiro - Copacabana,Copacabana,-22.963672,-43.178158,672.0,340659121,Tiago,within an hour,100%,...,6,1.0,1.0,4.0,"[""TV"", ""Laundromat nearby"", ""Exterior security...",0,NaN,NaN,NaN,NaN
39497,1319650312466603776,No coração de Ipanema…,Ipanema,-22.982799,-43.204781,1083.0,67244539,Caio,within an hour,100%,...,6,1.5,2.0,1.0,"[""Other gas stove"", ""Clothing storage: closet ...",0,NaN,NaN,NaN,NaN


In [ ]:
# Merge
df = pd.merge(df_listings, df_prices, on='listing_id', how='inner')

In [ ]:
df